In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [15]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
DOLLAR_THRESHOLD = "115009542m"

START_DATE = "20210101"
END_DATE = "20241231"

# Build base name
if SAMPLE_TYPE == "dollar":
    RESAMPLED_NAME = f"{SYMBOL}-{DOLLAR_THRESHOLD}-dollar-{START_DATE}-{END_DATE}"
else:
    RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"

In [16]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)

CPU times: user 1.28 ms, sys: 106 ms, total: 107 ms
Wall time: 107 ms


In [17]:
df.shape

(1487107, 91)

In [18]:
df.head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-04 01:24:00,103.1030,103.1105,103.1005,103.1095,1.864500e+11,0.003222,0.0080,0.000078,0.000078,0.000055,...,4,1,0.258819,0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 01:25:00,103.1095,103.1150,103.1000,103.1000,1.351400e+11,0.002707,-0.0095,-0.000092,-0.000092,0.000029,...,4,1,0.258819,0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 01:26:00,103.0985,103.0995,103.0915,103.0945,1.598100e+11,0.002979,-0.0055,-0.000053,-0.000053,-0.000008,...,4,1,0.258819,0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 01:27:00,103.0935,103.0975,103.0870,103.0925,1.293100e+11,0.002524,-0.0020,-0.000019,-0.000019,-0.000006,...,4,1,0.258819,0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 01:28:00,103.0930,103.1190,103.0930,103.1175,3.740000e+11,0.002553,0.0250,0.000243,0.000242,0.000031,...,4,1,0.258819,0.965926,0.0,1.0,0.724793,0.688967,0.5,0.866025


In [19]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_delta',
       'close_return', 'close_log_return', 'ret_mean_5', 'ret_mean_10',
       'ret_mean_15', 'ret_mean_20', 'log_volume', 'ema5', 'ema5_slope',
       'close_above_ema5', 'ema20', 'ema20_slope', 'close_above_ema20',
       'ema50', 'ema50_slope', 'close_above_ema50', 'ema100', 'ema100_slope',
       'close_above_ema100', 'atr14', 'atr25', 'atr50', 'log_atr14',
       'atr14_percent', 'atr14_adjusted_return', 'log_atr25', 'atr25_percent',
       'atr25_adjusted_return', 'log_atr50', 'atr50_percent',
       'atr50_adjusted_return', 'adx14', 'plus_di14', 'minus_di14', 'bb_upper',
       'bb_lower', 'bb_mavg', 'bb_width', 'bb_position', 'dc20_upper',
       'dc20_lower', 'dc20_mid', 'dc20_width', 'close_above_dc20_mid',
       'dc20_breakout', 'dc20_breakdown', 'dc50_upper', 'dc50_lower',
       'dc50_mid', 'dc50_width', 'close_above_dc50_mid', 'dc50_breakout',
       'dc50_breakdown', 'dc100_upper', 'dc100_lower', 

## Calculate Daily Volatility

In [20]:
vol = df['close_log_return'].rolling(window=60).std() # 計算 60 分鐘的 標準差
vol.mean()

np.float64(0.00012612278057731147)

## Get Events Start Time (t0)

In [21]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def get_tevents_optimized(data: pd.Series, threshold: float) -> pd.DatetimeIndex:
    # get tEvents using CUSUM filter
    values = data.values
    timestamps = data.index

    s_pos = np.zeros_like(values)
    s_neg = np.zeros_like(values)

    t_events_mask = np.zeros_like(values, dtype=bool)

    cum_pos, cum_neg = 0.0, 0.0

    for i in tqdm(range(len(values))):
        cum_pos = max(0.0, cum_pos + values[i])
        cum_neg = min(0.0, cum_neg + values[i])
        s_pos[i] = cum_pos
        s_neg[i] = cum_neg

        if cum_pos > threshold:
            t_events_mask[i] = True
            cum_pos = 0.0
        if cum_neg < -threshold:
            t_events_mask[i] = True
            cum_neg = 0.0

    return timestamps[t_events_mask]


In [22]:
EVENT_NAME = 'CUSUM'
THRESHOLD = vol.mean()*2
EVENT_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}_{THRESHOLD:.2e}.pkl"

In [23]:
%%time

t_events = get_tevents_optimized(df['close_log_return'].iloc[1:], threshold=THRESHOLD)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1487106/1487106 [00:00<00:00, 3463234.09it/s]

CPU times: user 430 ms, sys: 5.89 ms, total: 436 ms
Wall time: 436 ms


In [24]:
t_events

DatetimeIndex(['2021-01-04 01:35:00', '2021-01-04 01:40:00',
               '2021-01-04 01:43:00', '2021-01-04 01:45:00',
               '2021-01-04 01:50:00', '2021-01-04 02:01:00',
               '2021-01-04 02:23:00', '2021-01-04 02:41:00',
               '2021-01-04 02:45:00', '2021-01-04 02:52:00',
               ...
               '2024-12-30 22:29:00', '2024-12-30 23:00:00',
               '2024-12-30 23:06:00', '2024-12-30 23:10:00',
               '2024-12-30 23:12:00', '2024-12-30 23:17:00',
               '2024-12-30 23:18:00', '2024-12-30 23:35:00',
               '2024-12-30 23:40:00', '2024-12-30 23:47:00'],
              dtype='datetime64[ns]', name='timestamp', length=248970, freq=None)

In [25]:
EVENT_PATH

PosixPath('../data/events/USDJPY-1m-20210101-20241231_CUSUM_2.52e-04.pkl')

In [26]:
t_events.to_series().to_pickle(EVENT_PATH)